In [236]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import spacy
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import numpy as np
import nltk
import random
from langdetect import detect
from pathlib import Path
from textblob import TextBlob
from spacy.util import minibatch, compounding
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

In [99]:
stemmer = SnowballStemmer('french')

In [100]:
def lemmatise_text(text):
    tweet_nlp = nlp(text)
    l = [token.lemma_ for token in tweet_nlp ]
    return (" ".join(l)).replace(" \n","")

def stem_text(text):
    words = word_tokenize(text)
    l = [stemmer.stem(token) for token in words]
    return " ".join(l)


In [3]:
nlp = spacy.load("C:/Users/Benco/Anaconda3/Lib/site-packages/fr_core_news_sm/fr_core_news_sm-2.2.5")

In [20]:
train = pd.read_csv("train_data.csv")
train = train[["question" , "reponse"]]
train["lemmas_rep"] = train["reponse"].apply(lemmatise_text)
train["lemmas_quest"] = train["question"].apply(lemmatise_text)
train["stem_rep"] = train["reponse"].apply(stem_text)
train["stem_quest"] = train["question"].apply(stem_text)

In [21]:
train.to_csv("training_data.csv" , index = False)

In [22]:
train_random = (pd.read_csv("training_data.csv")).sample(120)
train_random.to_csv("training_data_random.csv" , index = False)

# Exécute à partir d'ici : il faut q'on ait le même fichier quand même

In [23]:
train_random = pd.read_csv("training_data_random.csv")


In [217]:
indice_test = [i for i in range(144) if(i not in train_random.index)] 
X_test = (train.iloc[indice_test , :][["question" , "reponse"]])
X_test["target"] = 1
X_test.columns = ["comb_quest" , "comb_rep" , "target"]

In [55]:
sw = nltk.corpus.stopwords.words('french')
sw += ['être', 'avoir']

In [70]:
vectorizer = TfidfVectorizer(lowercase=True, stop_words = sw,
                            ngram_range=(1, 1),
                             min_df = 1,
                            use_idf=True, smooth_idf=True, # idf lissé
                            sublinear_tf=False, norm='l2')

In [71]:
X_train_vectorized_tfidf = vectorizer.fit_transform(pd.concat([train_random["lemmas_quest"].apply(stem_text) , train_random["lemmas_rep"].apply(stem_text)]))# compléter ici

In [84]:
query = [stem_text("Quels sont les tarifs pour une réservation à l'hôtel ?")]
query_vector = vectorizer.transform(query)
pd.DataFrame(query_vector.toarray(), columns=vectorizer.get_feature_names())

,000,01,02,024,03,078,10,100,10h,11,...,émettr,équip,établ,étap,état,étendr,étroit,évad,éven,évit
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
query_corpus_sim = np.squeeze(cosine_similarity(X_train_vectorized_tfidf, query_vector)) ## on enlève leur format qui n'est pas bon --> vecteur
np.max(query_corpus_sim)

0.5597115400373813

In [92]:
id_best_doc = np.argsort(query_corpus_sim)[-1]
id_best_doc

92

In [94]:
 ## ordre croissant
if(id_best_doc >= 120):
    doc = train_random["reponse"][id_best_doc-120]
else:
    doc = train_random["reponse"][id_best_doc]
doc 

"Il existe un hôtel Disney pour toutes les bourses à Disneyland Paris. Vous pouvez découvrir les tarifs via notre service de réservation en ligne pratique et sécurisé.Trouvez votre hôtel de rêve DisneyRéservez votre séjour dès aujourd'hui !"

,comb_quest,comb_rep,target
120,puis - je réserv le mêm forf en lign et par té...,"malheur , non . notr servic de réserv en lign ...",1
121,comment postul pour un travail à disneyland pa...,disneyland paris être toujour à le recherch de...,1
122,qu ' y avoir t il à fair à disneyland paris ?,disneyland paris être un destin de vacanc magi...,1
123,puis - je apport de le nourritur dan le parc d...,le rep collect qui nécessit un logist particul...,1
124,pourquoi un messag de erreur se affich - t - i...,il pouvoir y avoir plusieur raison à cel : le ...,1
125,que signifi télécharg et comment télécharg un ...,télécharg être un procédur qui vous permettr d...,1
126,comment trouv rapid un inform sur un sujet par...,consult le plan de sit . il offrir un vu de en...,1
127,que être ce que le fastp disneyland paris et c...,le disneyland paris fastpass être un servic gr...,1
128,y avoir t il un acces wif à disneyland paris ?,oui ! le wif gratuit être disponibl dan certai...,1
129,être ce que je aller recevoir un offre de reno...,"lorsqu votr pass annuel arriv à expir , vous r...",1


In [218]:
html = requests.get("https://www.nike.com/fr/fr_fr/c/kids/how-to-buy-kids-shoes-faq").text
soup = BeautifulSoup(html, 'lxml')
classe_texte_nike = "nike-cq-subtitle-line-1 nike-cq-title-line nike-cq-line1 nsg-text--dark-grey nike-cq-font22px nike-cq-spacing08px nsg-font-family--platform"
questions_nike = [t.text for t in soup.find_all("span" , {"class" : classe_texte_nike})]
reponses_nike = [t.find("p").text.replace("\xa0" , "").replace("\n" , "").replace("  ","") for t in soup.find_all("div" , {"class" : "nike-cq-text-component nike-cq-misc-copy-block-text-resource"})]
questions_nike = (pd.Series(questions_nike))
reponses_nike = (pd.Series(reponses_nike))
df = pd.concat([questions_nike , reponses_nike] , axis = 1)
df["target"] = 0
df.columns = ["comb_quest" , "comb_rep" , "target"]

In [219]:
html = requests.get("https://mobile.free.fr/assistance/163.html").text
soup = BeautifulSoup(html, 'lxml')  
lq = pd.Series([t.text.replace("\xa0" , "") for t in soup.select("li.clicMe > strong")])
lr =[t.text.replace("\xa0" , "").replace("\n","").replace("\t","") for t in soup.find_all("li" , {"class" : "hideMe"})]
lr = (pd.Series(lr[:22]))
df_bis = pd.concat([lq, lr], axis=1)
df_bis["target"] = 0
df_bis.columns = ["comb_quest" , "comb_rep" , "target"]

In [220]:
df = df.append(df_bis)
df.index = list(range(len(df)))
X_test = X_test.append(df)
X_test.index = list(range(len(X_test)))

In [223]:
D = X_test[["comb_quest" , "comb_rep"]].apply(lambda textes : [stem_text(lemmatise_text(txt)) for txt in textes])
D["target"] = X_test["target"]
X_test = D #je sais que c'est laid !

In [249]:
liste_classif = []

for quest in X_test["comb_quest"].to_list():
    query_vector = vectorizer.transform([quest])
    query_corpus_sim = np.squeeze(cosine_similarity(X_train_vectorized_tfidf, query_vector))
    seuil = np.max(query_corpus_sim)
    if(seuil > 0.5):
        liste_classif.append(1)
    else:
        liste_classif.append(0)

res =  classification_report(X_test["target"], liste_classif)    
print(res)

              precision    recall  f1-score   support

           0       0.92      0.97      0.94        35
           1       0.95      0.88      0.91        24

    accuracy                           0.93        59
   macro avg       0.94      0.92      0.93        59
weighted avg       0.93      0.93      0.93        59



In [256]:
indice = [i for i in range(len(liste_classif)) if(liste_classif[i] != X_test["target"][i])]
indice

[3, 14, 15, 33]

In [259]:
[X_test["comb_quest"][3],
X_test["comb_quest"][14],
X_test["comb_quest"][15],
X_test["comb_quest"][33],
]

['puis - je apport de le nourritur dan le parc disney ?',
 'je être handicap . quel attract me conven ?',
 'quel attract conven à mon enfant ?',
 'que être ce que le col dynamic fair ?']

In [260]:
[X_test["comb_rep"][3],
X_test["comb_rep"][14],
X_test["comb_rep"][15],
X_test["comb_rep"][33],
]

['le rep collect qui nécessit un logist particuli ( glaci , tabl , emballag volumin ) être interdir dan le parc . un zon de pique-n être à votr disposit entre le parking et le entré un parc . vous pouvoir profit encor plus de votr séjour avec notr différent offre de restaur . plus de 65 restaur et bar pour tout le budget être propos avec un décor et un menu original . si vous craindr que notr sélect de option de restaur ne vous conven pas , consult notr servic dédi aux person avoir un besoin alimentair particuli .',
 "disneyland paris être dot de attract conven à tout le visiteur . beaucoup de entre lui être accessibl al fauteuil roul et d ' autr nécessit un transfert . certain attract être équip de boucl magnet pour le malentend . vous pouvoir découvr en lign quel attract vous conven . dan le parc , un symbol être présent à côt de chaqu attract , signal le limit ou restrict . pour plus de inform , report - vous al plan de le accessibil dan le parc disney . découvr tout le inform à con